In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
from collections import deque
import matplotlib.pylab as plt
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit



In [ ]:
df=pd.read_csv('../input/multijet-primary-dataset/MultiJetRun2010B.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T.sort_values(ascending =0,by='mean').style.background_gradient(cmap='BuGn').bar(subset=['std'], color='red').bar(subset=['mean'], color='blue')

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
correl=df.corr()
fig = plt.figure(figsize=(15,7))
sns.heatmap(correl,annot=True)

In [ ]:
df.columns

In [ ]:
df.plot(subplots=True, figsize=(10,12))
plt.title(' behavior')
plt.show()

In [ ]:
features=df.columns
for feature in features:
     print(f'{feature}--->{df[feature].nunique()}')

In [ ]:
df.nBJets.unique()

In [ ]:
df.nJets.unique()

In [ ]:
categorical_list = ["nJets"]
df_categoric = df.loc[:, categorical_list]
for i in categorical_list:
    plt.figure()
    sns.countplot(x = i, data = df_categoric,hue = "nJets")
    plt.title(i)

In [ ]:
df["nJets"].value_counts()

In [ ]:



sns.f, ax = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

#fig, axes = plt.subplots((2, 2),figsize=(15, 15))

#create chart in each subplot
#sns.boxplot(data=df, x='Run', y='nJets', ax=ax[0,0])
sns.boxplot(data=df, x='Run', ax=ax[0,0])
sns.boxplot(x='Lumi',data=df,  ax=ax[1,0])
sns.boxplot(x='Event',data=df,  ax=ax[2,0])
sns.boxplot(x='MR',data=df,  ax=ax[3,0])
sns.boxplot(x='Rsq',data=df,  ax=ax[0,1])
sns.boxplot(x='E1',data=df,  ax=ax[1,1])
sns.boxplot(x='Px1',data=df,  ax=ax[2,1])
sns.boxplot(x='Py1',data=df,  ax=ax[3,1])
sns.boxplot(x='Pz1',data=df,  ax=ax[0,2])
sns.boxplot(x='E2',data=df,  ax=ax[1,2])
sns.boxplot(x='Px2',data=df,  ax=ax[2,2])
sns.boxplot(x='Py2',data=df,  ax=ax[3,2])
sns.boxplot(x='Pz2',data=df,  ax=ax[0,3])
sns.boxplot(x='HT',data=df,  ax=ax[1,3])
sns.boxplot(x='MET',data=df,  ax=ax[2,3])
sns.boxplot(x='nJets',data=df,  ax=ax[3,3])


In [ ]:
sns.pairplot(df,palette='hls')
plt.show()

In [ ]:
X=df.drop(['nJets'],axis='columns')
y=df['nJets']

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=888, shrinkage=0.1)
X_m, y_m = ros.fit_resample(X, y)



In [ ]:
sns.countplot(y_m)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_m, y_m,test_size=0.2,random_state=5)

In [ ]:
from xgboost import XGBClassifier
from sklearn import svm
model_params = {
     
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini','entropy'],
        }    
    },
       'XGB_Classifier':{
        'model':XGBClassifier(),
        'params':{
            'base_score':[0.5]
            
        }
    }, 
        
     
      }


In [ ]:
from sklearn.model_selection import GridSearchCV
scores = []
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
reg=XGBClassifier()
reg.fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

In [ ]:
y_predicted = reg.predict(X_test)
y_predicted[:5]

In [ ]:
y_test[:5]

In [ ]:

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)